In [ ]:
import requests
import os
from dotenv import load_dotenv
from rag_agent_utils import parse_clu_response, build_kopis_query, make_gpt_prompt

# 1. 설정 불러오기
load_dotenv()

# 2. CLU 호출
clu_headers = {
    "Ocp-Apim-Subscription-Key": os.getenv("clu_key"),
    "Content-Type": "application/json"
}
clu_url = os.getenv("clu_endpoint")
query_text = input("입력하세요")

clu_body = {
    "kind": "Conversation",
    "analysisInput": {
        "conversationItem": {
            "id": "1",
            "participantId": "user",
            "text": query_text,
            "modality": "text",
            "language": "ko"
        }
    },
    "parameters": {
        "projectName": "PPCLU",
        "deploymentName": "ppv1-test",
        "verbose": True,
        "stringIndexType": "TextElement_V8"
    }
}

clu_response = requests.post(clu_url, headers=clu_headers, json=clu_body).json()

# 3. CLU 결과 → 검색 조건 생성
entities = parse_clu_response(clu_response)
kopis_query = build_kopis_query(entities)

# 4. (예시용) 검색 결과 가짜 생성
search_results = """
- 뮤지컬 <엘리자벳> @세종문화회관 (3/22~3/31)
- 뮤지컬 <레베카> @예술의전당 (3/25~4/3)
"""

# 5. 프롬프트 생성
prompt = make_gpt_prompt(entities, search_results)

# 6. GPT 호출 함수
def request_gpt(prompt):
    endpoint = os.getenv("oai_endpoint")
    headers = {
        "Content-Type": "application/json",
        "api-key": os.getenv("oai_key")
    }
    body = {
        "messages": [
            {
                "role": "system",
                "content": "공연 산업에 대한 지식이 아주 해박해서 이쪽 방면으로 질문이 들어오면 모든 것을 말해줄 수 있는 0.1% 전문가"
            },
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 1000,
        "data_sources": [
            {
                "type": "azure_search",
                "parameters": {
                    "endpoint": os.getenv("search_endpoint"),
                    "semantic_configuration": os.getenv("search_semanticname"),
                    "query_type": "semantic",
                    "fields_mapping": {},
                    "in_scope": True,
                    "filter": None,
                    "strictness": 3,
                    "top_n_documents": 5,
                    "key": os.getenv("search_key"),
                    "indexName": os.getenv("search_indexname")
                }
            }
        ]
    }

    response = requests.post(endpoint, headers=headers, json=body)
    if response.status_code == 200:
        response_json = response.json()
        content_text = response_json['choices'][0]['message']['content']
        citations = response_json['choices'][0]['message']['context']['citations']
        return content_text, citations
    else:
        return response.status_code, response.text

# 7. GPT 호출
response_text, citations = request_gpt(prompt)
print("GPT 응답:\n", response_text)
print("출처:\n", citations)


GPT 응답:
 400
출처:
 {"error": {"requestid": "2dddf3d4-da5f-4b07-b849-62f3df11923f", "code": 400, "message": "Validation error at #/data_sources/0/azure_search/parameters/endpoint: Assertion failed, \nValidation error at #/data_sources/0/azure_search/parameters/authentication: Field required\nValidation error at #/data_sources/0/azure_search/parameters/index_name: Field required\nValidation error at #/data_sources/0/azure_search/parameters/indexName: Extra inputs are not permitted"}}


In [ ]:
import os
import json
from dotenv import load_dotenv

# Add OpenAI import
from openai import AzureOpenAI

def main(): 
    try:
        # Flag to show citations
        show_citations = True

        # Get configuration settings 
        load_dotenv()
        azure_oai_endpoint = os.getenv("AZURE_OAI_ENDPOINT")
        azure_oai_key = os.getenv("AZURE_OAI_KEY")
        azure_oai_deployment = os.getenv("AZURE_OAI_DEPLOYMENT")
        azure_search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
        azure_search_key = os.getenv("AZURE_SEARCH_KEY")
        azure_search_index = os.getenv("AZURE_SEARCH_INDEX")
        
        # Initialize the Azure OpenAI client
        client = AzureOpenAI(
            base_url=azure_oai_endpoint,
            api_key=azure_oai_key,
            api_version="2024-02-15-preview"
        )

        # Get the prompt
        text = input('\nEnter a question:\n')

        # Configure your data source with corrected keys and type
        extension_config = {
            "data_sources": [
                {
                    "type": "azure_search",  
                    "parameters": {
                        "endpoint": azure_search_endpoint,
                        "index_name": azure_search_index,
                        "authentication": {
                            "type": "api_key",
                            "api_key": azure_search_key
                        }
                    }
                }
            ]
        }

        # Send request to Azure OpenAI model
        print("...Sending the following request to Azure OpenAI endpoint...")
        print("Request: " + text + "\n")

        response = client.chat.completions.create(
            model=azure_oai_deployment,
            temperature=0.5,
            max_tokens=1000,
            messages=[
                {"role": "system", "content": "You are a helpful travel agent"},
                {"role": "user", "content": text}
            ],
            extra_body=extension_config
        )

        # Print response
        print("Response: " + response.choices[0].message.content + "\n")

        if show_citations:
            # Print citations
            print("Citations:")
            citations = response.choices[0].message.context["messages"][0]["content"]
            citation_json = json.loads(citations)
            for c in citation_json["citations"]:
                print("  Title: " + c['title'] + "\n    URL: " + c['url'])

    except Exception as ex:
        print(ex)

if __name__ == '__main__': 
    main()


In [ ]:
print(type(azure_search_key))

In [ ]:
n = 0
for citation in request_result2:
    n+=1
    print(f'doc {n}:\n',citation['content'])
    print('*'*30)

In [ ]:
print(request_result['choices'][0]['message']['content'])

In [ ]:
request_result.status_code, request_result.json()